### Gold layer

In [0]:
spark.sql("""
          create schema if not exists workspace.Glod
          """)

In [0]:
spark.sql("""
          create volume if not exists workspace.Glod.ecommerce_data
          """)

In [0]:
silver = spark.read.format("delta") \
    .load("/Volumes/workspace/silver/ecommerce_data")


In [0]:
silver.display()

In [0]:
 import pyspark.sql.functions as F

###  GOLD: Aggregates

In [0]:


product_perf = (
    silver
    .withColumn("price_double", F.col("price").cast("double"))
    .groupBy("product_id", "category_code")
    .agg(
        F.countDistinct(
            F.when(F.col("event_type") == "view", F.col("user_id"))
        ).alias("views"),

        F.countDistinct(
            F.when(F.col("event_type") == "purchase", F.col("user_id"))
        ).alias("purchases"),

        F.sum(
            F.when(F.col("event_type") == "purchase", F.col("price_double"))
        ).alias("revenue")
    )
    .withColumn(
        "conversion_rate",
        F.when(
            F.col("views") > 0,
            (F.col("purchases") / F.col("views")) * 100
        ).otherwise(F.lit(0.0))
    )
)



In [0]:
display(product_perf)

In [0]:
product_perf.write.format("delta").mode("overwrite").save("/Volumes/workspace/Glod/ecommerce_data")

In [0]:
product_perf.count()